#### Import libraries and Enable GPU

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
from matplotlib import pyplot
import MNISTtools
import torch.utils.data as data_utils
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


#### Import features and labels to numpy arrays. 
#### Normalize images between [-1,1] 
#### Convert labels to one-hot encoding 

In [23]:
xtrain, ltrain = MNISTtools.load(dataset='training', path = '/datasets/MNIST')
xtrain = np.transpose(xtrain)

def normalize_MNIST_images(x):
    x = x.astype(np.float32)
    x = (x - 127.5)/127.5
    return x
xtrain = normalize_MNIST_images(xtrain)

def label2onehot(lbl):
    d = np.zeros((lbl.max() + 1, lbl.size))
    d[lbl[np.arange(0, lbl.size)], np.arange(0, lbl.size)] = 1
    return d
dtrain = label2onehot(ltrain)

dtrain = np.transpose(dtrain)
print(xtrain.shape)
print(dtrain.shape)

(60000, 784)
(60000, 10)


#### Load numpy arrays as tensors onto the GPU

In [24]:
img = torch.from_numpy(xtrain).float().to(device)
lbl = torch.from_numpy(dtrain).float().to(device)
'''Below is for mini batch gradient decent '''
# train = data_utils.TensorDataset(img,lbl)
# data_loader = data_utils.DataLoader(train, batch_size = batch_size, shuffle = True)
# for i, (images, labels) in enumerate(data_loader):
#     print(images.size(),labels.size())
print(img.size())
print(lbl.size())
print(type(img))
print(type(lbl))

torch.Size([60000, 784])
torch.Size([60000, 10])
<class 'torch.Tensor'>
<class 'torch.Tensor'>


#### Set the hyper parameters and define cross entropy loss and Xavier Initialization

In [29]:
#Hyperparameters
input_size = 784
hidden_size = 64
num_classes = 10
num_epochs = 200
batch_size = 100
learning_rate = 0.001

def cross_entropy(input, target):
    logsoftmax = nn.LogSoftmax(dim = 1)
    return torch.mean(torch.mean(-target * logsoftmax(input), dim=1))

def Network_Initialization(model):
    if type(model) == nn.Linear:
        torch.nn.init.xavier_uniform_(model.weight)
        model.bias.data.fill_(0.01)

#### Initialize the model 

In [30]:
model = torch.nn.Sequential(nn.Linear(input_size, hidden_size),nn.ReLU(),nn.Linear(hidden_size, num_classes),nn.Softmax()).to(device)
model.apply(Network_Initialization)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

#### Define miss classifications 

In [31]:
yinit = model(img)
print("Loss is", cross_entropy(yinit, lbl).item())

def miss_classifications(yinit, lbl):
    _, predicted = torch.max(yinit.data, 1)
    miss_classifications = (predicted != torch.max(lbl.data, 1)[1]).sum().item()/600
    return miss_classifications

print(miss_classifications(yinit, lbl)  )

('Loss is', 0.2304781973361969)
89


#### Model Training 

In [32]:
for epoch in range(num_epochs):
    
    # Forward pass
    outputs = model(img)
    loss = cross_entropy(outputs,lbl)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    mc = miss_classifications(outputs, lbl)
    print ('Epoch: {}, Miss-Classifications: {}, Loss: {}'.format(epoch+1, mc , loss.item()))

torch.cuda.empty_cache()

Epoch: 1, Miss-Classifications: 89, Loss: 0.230478197336
Epoch: 2, Miss-Classifications: 76, Loss: 0.226385489106
Epoch: 3, Miss-Classifications: 74, Loss: 0.222975283861
Epoch: 4, Miss-Classifications: 68, Loss: 0.220030754805
Epoch: 5, Miss-Classifications: 66, Loss: 0.217205613852
Epoch: 6, Miss-Classifications: 64, Loss: 0.214683324099
Epoch: 7, Miss-Classifications: 63, Loss: 0.212598532438
Epoch: 8, Miss-Classifications: 62, Loss: 0.210689052939
Epoch: 9, Miss-Classifications: 61, Loss: 0.209145516157
Epoch: 10, Miss-Classifications: 59, Loss: 0.207376554608
Epoch: 11, Miss-Classifications: 55, Loss: 0.205486416817
Epoch: 12, Miss-Classifications: 51, Loss: 0.203228294849
Epoch: 13, Miss-Classifications: 49, Loss: 0.201692417264
Epoch: 14, Miss-Classifications: 48, Loss: 0.200155705214
Epoch: 15, Miss-Classifications: 47, Loss: 0.198153659701
Epoch: 16, Miss-Classifications: 47, Loss: 0.197091847658
Epoch: 17, Miss-Classifications: 46, Loss: 0.195970028639
Epoch: 18, Miss-Classif

Epoch: 144, Miss-Classifications: 17, Loss: 0.164559990168
Epoch: 145, Miss-Classifications: 17, Loss: 0.164535671473
Epoch: 146, Miss-Classifications: 17, Loss: 0.164511725307
Epoch: 147, Miss-Classifications: 16, Loss: 0.164488002658
Epoch: 148, Miss-Classifications: 16, Loss: 0.16446454823
Epoch: 149, Miss-Classifications: 16, Loss: 0.164441376925
Epoch: 150, Miss-Classifications: 16, Loss: 0.164418444037
Epoch: 151, Miss-Classifications: 16, Loss: 0.164395779371
Epoch: 152, Miss-Classifications: 16, Loss: 0.16437330842
Epoch: 153, Miss-Classifications: 16, Loss: 0.164351120591
Epoch: 154, Miss-Classifications: 16, Loss: 0.164329171181
Epoch: 155, Miss-Classifications: 16, Loss: 0.164307489991
Epoch: 156, Miss-Classifications: 16, Loss: 0.164285987616
Epoch: 157, Miss-Classifications: 16, Loss: 0.164264693856
Epoch: 158, Miss-Classifications: 16, Loss: 0.164243683219
Epoch: 159, Miss-Classifications: 16, Loss: 0.164222836494
Epoch: 160, Miss-Classifications: 16, Loss: 0.164202228189